In [ ]:
import PyPDF2

def pdf_to_text(pdf_path, output_text_file):
    with open(pdf_path, 'rb') as pdf_file:
        pdf_reader = PyPDF2.PdfReader(pdf_file)

        metin = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            metin += page.extract_text()

    with open(output_text_file, 'w', encoding='utf-8') as output_file:
        output_file.write(metin)

if __name__ == "__main__":
    pdf_path = "yasarkemal.pdf"  # PDF dosyanızın yolunu belirtin
    output_text_file = "roman.txt"

    pdf_to_text(pdf_path, output_text_file)


In [11]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, RepeatVector, TimeDistributed, Dense

# Dosyadan metni okuma
with open("roman2.txt", "r", encoding="utf-8") as file:
    metin = file.read()

# Metni küçük harfe çevirme
metin = metin.lower()

# Karakter setini oluşturma
characters = sorted(set(metin))
char_to_index = {char: idx for idx, char in enumerate(characters)}
index_to_char = {idx: char for idx, char in enumerate(characters)}

# Veriyi hazırlama: Giriş ve çıkış olarak aynı metni kullanıyoruz
maxlen = 40  # Her örnek için kullanılacak maksimum karakter sayısı
step = 3    # Her adımda alınacak karakter sayısı

sentences = []  # Giriş metinleri
next_chars = []  # Çıkış metinleri

for i in range(0, len(metin) - maxlen, step):
    sentences.append(metin[i:i + maxlen])
    next_chars.append(metin[i + maxlen])

# Veriyi numpy dizilerine çevirme
x = np.zeros((len(sentences), maxlen, len(characters)), dtype=bool)
y = np.zeros((len(sentences), maxlen, len(characters)), dtype=bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1
        y[i, t, char_to_index[next_chars[i]]] = 1

# LSTM tabanlı otoencoder modeli oluşturma
model = Sequential([
    LSTM(128, input_shape=(maxlen, len(characters))),  # LSTM katmanı, 128 hücre ve giriş boyutu (maxlen, len(characters))
    RepeatVector(maxlen),  # Çıkış vektörünü (maxlen) kadar tekrar etme
    LSTM(128, return_sequences=True),  # LSTM katmanı, 128 hücre ve zaman serilerini döndürme (return_sequences=True)
    TimeDistributed(Dense(len(characters), activation='softmax'))  # Her zaman adımındaki dağılım katman
])

model.compile(optimizer='adam', loss='categorical_crossentropy')

# Modeli eğitme
model.fit(x, y, epochs=50, batch_size=32)






Epoch 1/50
28/28 [==============================] - 12s 88ms/step - loss: 3.5620
Epoch 2/50
28/28 [==============================] - 2s 82ms/step - loss: 3.2950
Epoch 3/50
28/28 [==============================] - 2s 74ms/step - loss: 3.2746
Epoch 4/50
28/28 [==============================] - 2s 72ms/step - loss: 3.2750
Epoch 5/50
28/28 [==============================] - 2s 82ms/step - loss: 3.2660
Epoch 6/50
28/28 [==============================] - 2s 84ms/step - loss: 3.2630
Epoch 7/50
28/28 [==============================] - 2s 89ms/step - loss: 3.2592
Epoch 8/50
28/28 [==============================] - 2s 83ms/step - loss: 3.2554
Epoch 9/50
28/28 [==============================] - 3s 91ms/step - loss: 3.2440
Epoch 10/50
28/28 [==============================] - 2s 84ms/step - loss: 3.2335
Epoch 11/50
28/28 [==============================] - 2s 84ms/step - loss: 3.1787
Epoch 12/50
28/28 [==============================] - 3s 95ms/step - loss: 3.0614
Epoch 13/50
28/28 [=================

In [12]:
# Rastgele bir başlangıç cümlesi seçme
start_index = np.random.randint(0, len(metin) - maxlen - 1)
seed_text = metin[start_index:start_index + maxlen]

# Metin üretme
generated_text = seed_text
for _ in range(40):
    x_pred = np.zeros((1, maxlen, len(characters)))
    for t, char in enumerate(seed_text):
        x_pred[0, t, char_to_index[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = np.argmax(preds[-1])
    next_char = index_to_char[next_index]

    generated_text += next_char
    seed_text = seed_text[1:] + next_char

print("Üretilen Metin:")
print(generated_text)



Üretilen Metin:
ası'nın kuruluşuna kahldı ve 1974-75 arasında iliri'i isın küriliği isarararır k
